In [ ]:
import numpy as np
import asyncio
from deep_orderbook.shaper import ArrayShaper
from deep_orderbook.config import ReplayConfig, ShaperConfig

In [ ]:
replay_conf_3 = ReplayConfig(
    markets=["ETH-USD", "BTC-USD", "ETH-BTC"],
    data_dir='data',
    date_regexp='2024-08-04',
    max_samples=-1,
)

In [ ]:
from deep_orderbook.replayer import iter_sec

async for onesec in iter_sec(config=replay_conf_3.but(max_samples=3)):
    print(onesec)

In [ ]:
shaper_config = ShaperConfig()


async def iter_shapes(config: ReplayConfig):
    symbol_shapers = {
        pair: ArrayShaper(config=shaper_config) for pair in config.markets
    }
    async for onesec in iter_sec(config=config):
        yield {
            market: await symbol_shapers[market].make_arr3d(onesec.symbols[market])
            for market in config.markets
        }


async for shaped in iter_shapes(
    config=replay_conf_3.but(max_samples=1, skip_until_time="23:15")
):
    for k, v in shaped.items():
        print(f"{k}: {v.shape}, values from {v.min()} to {v.max()}")

In [ ]:
from deep_orderbook.visu import Visualizer
import seaborn_image as isns
import matplotlib.pyplot as plt
from IPython.display import clear_output


async for shapes in iter_shapes(
    config=replay_conf_3.but(max_samples=100, skip_until_time="23:15")
):
    books_array = shapes['ETH-USD']
    im_data, _, _ = Visualizer.for_image_display(books_array=books_array)
    isns.imshow(im_data, origin='lower', cmap='coolwarm', vmin=-1, vmax=1, gray=True)
    clear_output(wait=True)
    plt.show()

In [ ]:
from deep_orderbook.shaper import iter_shapes_t2l

async for shapes in iter_shapes_t2l(
    replay_config=replay_conf_3.but(max_samples=1, skip_until_time="23:15"),
    shaper_config=shaper_config,
):
    print(shapes[0].shape, shapes[1].shape, shapes[2].shape)

In [ ]:
from deep_orderbook.visu import Visualizer

vis = Visualizer()


# Define your asynchronous function to update the figure
async def rolling_plot(config: ReplayConfig):
    async for books_array, t2l_array, pxar in iter_shapes_t2l(
        replay_config=config, shaper_config=shaper_config
    ):
        vis.update(books_z_data=books_array, level_reach_z_data=t2l_array, bidask=pxar)


# Run the asynchronous function
await rolling_plot(config=replay_conf_3.but(max_samples=100, skip_until_time="23:15"))

In [ ]:
import holoviews as hv
import panel as pn
from holoviews.streams import Pipe
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Enable the Bokeh backend for HoloViews and initialize Panel
hv.extension('bokeh')
pn.extension()

# Initialize Streams with appropriate empty data
im_pipe = Pipe(data=np.zeros((10, 10)))  # For 'Books' heatmap
t2l_pipe = Pipe(data=np.zeros((10, 10)))  # For 'Level Proximity' heatmap
bidask_pipe = Pipe(data=np.empty((0, 2)))  # For 'Bid and Ask Price Levels'

# Define HoloViews DynamicMaps for heatmaps
im_dmap = hv.DynamicMap(
    lambda data: hv.Image(data).opts(
        invert_yaxis=True,
        cmap='RdBu',
        colorbar=True,
        width=1200,
        height=200,
        title='Books',
        xaxis=None,
        yaxis=None,
        ylim=(-0.5, 0.5),
        axiswise=True,
        framewise=True,
        shared_axes=False,
    ),
    streams=[im_pipe],
)

t2l_dmap = hv.DynamicMap(
    lambda data: hv.Image(data).opts(
        invert_yaxis=True,
        cmap='Turbo',
        colorbar=True,
        width=1200,
        height=200,
        title='Level Proximity',
        xaxis=None,
        yaxis=None,
        ylim=(-0.5, 0.5),
        axiswise=True,
        framewise=True,
        shared_axes=False,
    ),
    streams=[t2l_pipe],
)


def bidask_plot(data):
    bid_curve = hv.Curve((data[:, 0]), 'bid').opts(
        color='green', line_width=2, framewise=True
    )
    ask_curve = hv.Curve((data[:, 1]), 'ask').opts(
        color='red', line_width=2, framewise=True
    )
    return (bid_curve * ask_curve).opts(
        width=1200,
        height=200,
        title='Bid and Ask',
        # yaxis=None, xaxis='bottom'
    )


bidask_dmap = hv.DynamicMap(bidask_plot, streams=[bidask_pipe])

# Layout the plots vertically using Panel
dashboard = pn.Column(pn.panel(im_dmap), pn.panel(t2l_dmap), pn.panel(bidask_dmap))


# Define the async update function
async def run_updates():
    async for shaped, t2l, pxar in iter_shapes_t2l(
        replay_config=replay_conf_3.but(max_samples=100, skip_until_time="23:15"),
        shaper_config=shaper_config,
    ):
        books_z_data, level_reach_z_data, bidask = Visualizer.for_image_display(
            shaped, t2l, pxar
        )
        im_pipe.send(np.clip(books_z_data, -1, 1))
        t2l_pipe.send(level_reach_z_data)
        bidask_pipe.send(bidask)


display(dashboard)
await asyncio.sleep(0.1)

# Start the async update in the background
asyncio.create_task(run_updates())
# dashboard.show()

In [ ]:
import holoviews as hv

hv.extension("bokeh", logo=False)

import numpy as np
import asyncio
import panel as pn

pn.extension()
import nest_asyncio

nest_asyncio.apply()


# Create a reactive HoloViews stream
stream = hv.streams.Stream.define('ImageUpdate', data=None)()


# Function to update the image
async def update_image():
    async for shapes in iter_shapes(
        config=replay_conf_3.but(max_samples=1000, skip_until_time="23:15")
    ):
        books_array = shapes['ETH-USD']
        im_display, _, _ = Visualizer.for_image_display(books_array=books_array)

        # Update the stream with the processed image
        stream.event(data=im_display)
        # await asyncio.sleep(0.1)


# Start the asynchronous update
asyncio.create_task(update_image())


# Function to create the hv.Image with correct bounds
def create_image(data):
    if data is not None:
        height, width = data.shape
        return hv.Image(data, bounds=(0, 0, width, height))
    else:
        # Placeholder image if data is None
        return hv.Image(np.zeros((128, 256)), bounds=(0, 0, 256, 128))


# DynamicMap that updates with the stream
dmap = hv.DynamicMap(create_image, streams=[stream])

# Set display options
dmap.opts(
    cmap='RdBu',
    frame_width=800,
    aspect=256 / 128,
    invert_yaxis=True,
    xlabel='second',
    ylabel='levels',
    colorbar=True,
    clim=(-1, 1),
)

# Display with Panel
pn.panel(dmap).show()

In [ ]:
import plotly.express as px
from skimage import io

img = io.imread(
    'https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Crab_Nebula.jpg/240px-Crab_Nebula.jpg'
)
fig = px.imshow(img)
fig.show()